In [145]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
import re
from datetime import datetime, timezone,timedelta
from dateutil import parser
from nba_api.stats.endpoints import scoreboardv2, ScoreboardV2
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.live.nba.endpoints import boxscore, BoxScore
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

In [146]:
yesterday = datetime.today() - timedelta(days=1) 
yesterday = yesterday.strftime("%Y-%m-%d")


In [147]:
lst = []
z = leaguegamefinder.LeagueGameFinder(date_from_nullable="2025-10-21",
                                      date_to_nullable=yesterday,
                                      season_type_nullable="Regular Season",
                                      league_id_nullable="00")
z = z.get_data_frames()[0]


In [148]:
x = pd.DataFrame(z)

In [149]:
x["TotalFGMade"] = x.groupby('GAME_ID')['FGM'].transform('sum')
x["TotalFGAtt"] = x.groupby('GAME_ID')['FGA'].transform('sum')
x['Total3PM'] = x.groupby('GAME_ID')['FG3M'].transform('sum')
x["TotalTOV"] = x.groupby('GAME_ID')['TOV'].transform('sum')
x["TotalFTA"] = x.groupby('GAME_ID')['FTA'].transform('sum')
x['TotalOREB'] = x.groupby('GAME_ID')['OREB'].transform('sum')
x['TotalGamePoints'] = x.groupby('GAME_ID')['PTS'].transform('sum')
x['OppOREB'] = x['TotalOREB'] - x['OREB']
x['TDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
x['TotalGameFTM'] = x.groupby('GAME_ID')['FTM'].transform('sum')
x['TotalDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
x['TotalAST'] = x.groupby('GAME_ID')['AST'].transform('sum')
x['TotalSTL'] = x.groupby('GAME_ID')['STL'].transform('sum')
x['TotalBLK'] = x.groupby('GAME_ID')['BLK'].transform('sum')
x['TotalPF'] = x.groupby('GAME_ID')['PF'].transform('sum')
x['TotalTOV'] = x.groupby('GAME_ID')['TOV'].transform('sum')

In [150]:
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 4)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return round(float(x['OREB']) / (float(x['OREB'] + x['OPPDREB'])),4)
def FTRATE(x):
    return round(float(x['FTA']) / float(x['FGA']),4)
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 4)
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 4)
def DREBP(x):
    return round(float(x['DREB']) / float(x['DREB'] + x['OppOREB']),4)
def OppFTRATE(x):
    return round(float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA']),4)
def away(x):
    return x["MATCHUP"][:4]
def home(x):
    return x["MATCHUP"][-4:]
def teamPIE(x):
    teampie = (x['PTS'] + x['FGM'] + x['FTM'] - x['FGA'] - x['FTA'] + x['DREB'] + (0.5 * x['OREB']) + x['AST'] + x['STL'] + (x['BLK']*0.5) - x['PF'] - x['TOV']) 
    gamepie = (x['TotalGamePoints'] + x['TotalFGMade'] + x['TotalGameFTM'] - x['TotalFGAtt'] - x['TotalFTA'] + x['TotalDREB'] + (0.5 * x['TotalOREB']) + x['TotalAST'] + x['TotalSTL'] + (0.5 * x['TotalBLK']) - x['TotalPF'] - x['TotalTOV'])
    return round(float(teampie) / float(gamepie), 4)


In [151]:
x['EFG'] = x.apply(EFG, axis=1)
x['TOVP'] = x.apply(TOVP, axis=1)
x['OREBP'] = x.apply(OREBP, axis=1)
x['FTRATE'] = x.apply(FTRATE, axis=1)
x['OppEFG'] = x.apply(OppEFG, axis=1)
x['OppTOVP'] = x.apply(OppTOVP, axis=1)
x['DREBP'] = x.apply(DREBP, axis=1)
x['OppFTRATE'] = x.apply(OppFTRATE, axis=1)
x["AWAY"] = x.apply(away, axis=1)
x["HOME"] = x.apply(home, axis=1)
x['TEAMPIE'] = x.apply(teamPIE, axis=1)

In [152]:
x.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)

In [153]:
def combine_game_rows(df):
    df = df.copy()
    df['is_home'] = df['MATCHUP'].str.contains('vs.', na=False)
    stat_columns = [col for col in df.columns if col not in 
                   ['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 
                    'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'AWAY', 'HOME']]
    home_data = df[df['is_home'] == True].copy()
    away_data = df[df['is_home'] == False].copy()
    home_rename = {col: f'home_{col}' for col in stat_columns}
    home_rename.update({
        'TEAM_ID': 'home_TEAM_ID',
        'TEAM_ABBREVIATION': 'home_TEAM',
        'TEAM_NAME': 'home_TEAM_NAME',
        'WL': 'home_WL'
    })
    home_data = home_data.rename(columns=home_rename)
    away_rename = {col: f'away_{col}' for col in stat_columns}
    away_rename.update({
        'TEAM_ID': 'away_TEAM_ID',
        'TEAM_ABBREVIATION': 'away_TEAM',
        'TEAM_NAME': 'away_TEAM_NAME',
        'WL': 'away_WL'
    })
    away_data = away_data.rename(columns=away_rename)
    merge_cols = ['GAME_ID', 'GAME_DATE']
    if 'SEASON_ID' in df.columns:
        merge_cols.append('SEASON_ID')
    combined = pd.merge(
        home_data[merge_cols + [col for col in home_data.columns if col not in merge_cols]],
        away_data[merge_cols + [col for col in away_data.columns if col not in merge_cols]],
        on=merge_cols,
        how='inner'
    )
    combined['home_win'] = (combined['home_WL'] == 'W').astype(int)
    return combined

In [154]:
x['_original_index'] = range(len(x))
x_sorted = x.sort_values(['TEAM_ID', 'SEASON_ID', 'GAME_DATE']).reset_index(drop=True)
ewma_cols = ['PTS', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
'OREB', 'DREB', 'AST', 'STL', 'TOV', 'PF', 'PLUS_MINUS', 'EFG', 'TOVP',
'OREBP', 'FTRATE', 'OppEFG', 'OppTOVP', 'DREBP', 'OppFTRATE', 'TEAMPIE']
for col in ewma_cols:
    x_sorted[f"EWMA {col}"] = (
    x_sorted.groupby(['TEAM_ID', 'SEASON_ID'])[col].apply(lambda x: x.shift(1).ewm(span=7, min_periods=7).mean().round(4)).reset_index(level=[0, 1], drop=True))
x_sorted = x_sorted.sort_values('_original_index').reset_index(drop=True)
x_sorted = x_sorted.drop(columns=['_original_index'])

In [ ]:
x_sorted = combine_game_rows(x_sorted)

In [ ]:
x_sorted.drop(columns=['AWAY_y','HOME_y', 'away_is_home','home_is_home','AWAY_x', 'HOME_x',], inplace=True)

In [156]:
x_sorted

,GAME_ID,GAME_DATE,SEASON_ID,home_TEAM_ID,home_TEAM,home_TEAM_NAME,MATCHUP_x,home_WL,home_MIN,home_PTS,home_FGM,home_FGA,home_FG_PCT,home_FG3M,home_FG3A,home_FG3_PCT,home_FTM,home_FTA,home_FT_PCT,home_OREB,home_DREB,home_REB,home_AST,home_STL,home_BLK,home_TOV,home_PF,home_PLUS_MINUS,home_TotalGamePoints,home_TotalGameFTM,home_TotalDREB,home_TotalAST,home_TotalSTL,home_TotalBLK,home_TotalPF,home_EFG,home_TOVP,home_OREBP,home_FTRATE,home_OppEFG,home_OppTOVP,home_DREBP,home_OppFTRATE,AWAY_x,HOME_x,home_TEAMPIE,home_EWMA PTS,home_EWMA FGA,home_EWMA FG_PCT,home_EWMA FG3M,home_EWMA FG3A,home_EWMA FG3_PCT,home_EWMA FTM,home_EWMA FTA,home_EWMA FT_PCT,home_EWMA OREB,home_EWMA DREB,home_EWMA AST,home_EWMA STL,home_EWMA TOV,home_EWMA PF,home_EWMA PLUS_MINUS,home_EWMA EFG,home_EWMA TOVP,home_EWMA OREBP,home_EWMA FTRATE,home_EWMA OppEFG,home_EWMA OppTOVP,home_EWMA DREBP,home_EWMA OppFTRATE,home_EWMA TEAMPIE,home_is_home,away_TEAM_ID,away_TEAM,away_TEAM_NAME,MATCHUP_y,away_WL,away_MIN,away_PTS,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FTM,away_FTA,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_STL,away_BLK,away_TOV,away_PF,away_PLUS_MINUS,away_TotalGamePoints,away_TotalGameFTM,away_TotalDREB,away_TotalAST,away_TotalSTL,away_TotalBLK,away_TotalPF,away_EFG,away_TOVP,away_OREBP,away_FTRATE,away_OppEFG,away_OppTOVP,away_DREBP,away_OppFTRATE,HOME_y,away_TEAMPIE,away_EWMA PTS,away_EWMA FGA,away_EWMA FG_PCT,away_EWMA FG3M,away_EWMA FG3A,away_EWMA FG3_PCT,away_EWMA FTM,away_EWMA FTA,away_EWMA FT_PCT,away_EWMA OREB,away_EWMA DREB,away_EWMA AST,away_EWMA STL,away_EWMA TOV,away_EWMA PF,away_EWMA PLUS_MINUS,away_EWMA EFG,away_EWMA TOVP,away_EWMA OREBP,away_EWMA FTRATE,away_EWMA OppEFG,away_EWMA OppTOVP,away_EWMA DREBP,away_EWMA OppFTRATE,away_EWMA TEAMPIE,away_is_home,home_win
0,0022500210,2025-11-11,22025,1610612760,OKC,Oklahoma City Thunder,OKC vs. GSW,W,240,126,42,83,0.506,16,36,0.444,26,31,0.839,10,30,40,29,12,4,19,21,24.0,228,49,56,52,28,8,49,0.6024,0.1643,0.2778,0.3735,0.5338,0.1873,0.8333,0.3919,OKC,GSW,0.6051,122.2753,91.6823,0.4761,13.5950,40.3459,0.3392,21.6928,25.1221,0.8625,9.7412,38.8787,24.6220,10.3161,10.0560,20.0371,16.3581,0.5508,0.0891,0.2193,0.2740,0.5024,0.1379,0.7873,0.2586,0.5870,True,1610612744,GSW,Golden State Warriors,GSW @ OKC,L,241,102,33,74,0.446,13,35,0.371,23,29,0.793,6,26,32,23,16,4,20,28,-24.0,228,49,56,52,28,8,49,0.5338,0.1873,0.1667,0.3919,0.6024,0.1643,0.7222,0.3735,OKC,0.3949,112.6197,88.5384,0.4455,14.3387,42.5288,0.3378,19.5079,24.7678,0.7988,11.2823,33.7009,26.5281,8.4533,13.9886,19.6012,3.7199,0.5272,0.1232,0.2414,0.2822,0.5227,0.1303,0.7645,0.2377,0.5221,False,1
1,0022500211,2025-11-11,22025,1610612762,UTA,Utah Jazz,UTA vs. IND,W,241,152,51,92,0.554,19,42,0.452,31,35,0.886,13,32,45,38,3,3,14,27,24.0,280,57,52,67,9,9,53,0.6576,0.1153,0.3939,0.3804,0.5152,0.0668,0.6809,0.2929,UTA,IND,0.5868,107.4835,90.2443,0.4056,12.1944,41.1549,0.2920,22.2075,27.3155,0.8103,14.6208,32.1523,27.1551,6.9709,16.1262,21.9327,-14.0903,0.4730,0.1361,0.2983,0.3051,0.5647,0.1172,0.7527,0.2745,0.4313,True,1610612754,IND,Indiana Pacers,IND @ UTA,L,240,128,46,99,0.465,10,37,0.270,26,29,0.897,15,20,35,29,6,6,8,26,-24.0,280,57,52,67,9,9,53,0.5152,0.0668,0.3191,0.2929,0.6576,0.1153,0.6061,0.3804,UTA,0.4132,100.6614,94.5405,0.3816,10.7283,37.7107,0.2838,17.8347,23.9560,0.7294,14.6640,34.7602,22.8710,7.1919,14.6127,23.5160,-15.0112,0.4383,0.1222,0.2782,0.2551,0.5227,0.1154,0.7539,0.3401,0.4071,False,1
2,0022500212,2025-11-11,22025,1610612758,SAC,Sacramento Kings,SAC vs. DEN,L,240,108,41,85,0.482,10,33,0.303,16,23,0.696,9,26,35,31,4,3,10,23,-14.0,230,37,59,55,10,5,42,0.5412,0.0951,0.2143,0.2706,0.5872,0.1092,0.6341,0.3140,SAC,DEN,0.4428,115.4702,88.6470,0.4664,12.0059,30.3964,0.3974,20.9260,26.3738,0.7764,9.2868,32.2401,26.3358,7.6551,14.5981,20.0546,-14.4707,0.5340,0.1269,0.2070,0.3000,0.5983,0.1070,0.7735,0.2648,0.4371,True,1610612743,DEN,Denver Nuggets,DEN @ SAC,W,239,122,44,86,0.512,13,33,0.394,21,27,0.778,1

In [157]:
x_sorted.to_sql("2025-2026 Team Data Combined", conn, if_exists="replace", index=False)

161

In [158]:
file = 'nba.db'
table = "[2025-2026 Team Data Combined]"

query = f"SELECT * FROM {table}"
x = pd.read_sql_query(query, conn)

In [159]:
x

,GAME_ID,GAME_DATE,SEASON_ID,home_TEAM_ID,home_TEAM,home_TEAM_NAME,MATCHUP_x,home_WL,home_MIN,home_PTS,home_FGM,home_FGA,home_FG_PCT,home_FG3M,home_FG3A,home_FG3_PCT,home_FTM,home_FTA,home_FT_PCT,home_OREB,home_DREB,home_REB,home_AST,home_STL,home_BLK,home_TOV,home_PF,home_PLUS_MINUS,home_TotalGamePoints,home_TotalGameFTM,home_TotalDREB,home_TotalAST,home_TotalSTL,home_TotalBLK,home_TotalPF,home_EFG,home_TOVP,home_OREBP,home_FTRATE,home_OppEFG,home_OppTOVP,home_DREBP,home_OppFTRATE,AWAY_x,HOME_x,home_TEAMPIE,home_EWMA PTS,home_EWMA FGA,home_EWMA FG_PCT,home_EWMA FG3M,home_EWMA FG3A,home_EWMA FG3_PCT,home_EWMA FTM,home_EWMA FTA,home_EWMA FT_PCT,home_EWMA OREB,home_EWMA DREB,home_EWMA AST,home_EWMA STL,home_EWMA TOV,home_EWMA PF,home_EWMA PLUS_MINUS,home_EWMA EFG,home_EWMA TOVP,home_EWMA OREBP,home_EWMA FTRATE,home_EWMA OppEFG,home_EWMA OppTOVP,home_EWMA DREBP,home_EWMA OppFTRATE,home_EWMA TEAMPIE,home_is_home,away_TEAM_ID,away_TEAM,away_TEAM_NAME,MATCHUP_y,away_WL,away_MIN,away_PTS,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FTM,away_FTA,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_STL,away_BLK,away_TOV,away_PF,away_PLUS_MINUS,away_TotalGamePoints,away_TotalGameFTM,away_TotalDREB,away_TotalAST,away_TotalSTL,away_TotalBLK,away_TotalPF,away_EFG,away_TOVP,away_OREBP,away_FTRATE,away_OppEFG,away_OppTOVP,away_DREBP,away_OppFTRATE,HOME_y,away_TEAMPIE,away_EWMA PTS,away_EWMA FGA,away_EWMA FG_PCT,away_EWMA FG3M,away_EWMA FG3A,away_EWMA FG3_PCT,away_EWMA FTM,away_EWMA FTA,away_EWMA FT_PCT,away_EWMA OREB,away_EWMA DREB,away_EWMA AST,away_EWMA STL,away_EWMA TOV,away_EWMA PF,away_EWMA PLUS_MINUS,away_EWMA EFG,away_EWMA TOVP,away_EWMA OREBP,away_EWMA FTRATE,away_EWMA OppEFG,away_EWMA OppTOVP,away_EWMA DREBP,away_EWMA OppFTRATE,away_EWMA TEAMPIE,away_is_home,home_win
0,0022500210,2025-11-11,22025,1610612760,OKC,Oklahoma City Thunder,OKC vs. GSW,W,240,126,42,83,0.506,16,36,0.444,26,31,0.839,10,30,40,29,12,4,19,21,24.0,228,49,56,52,28,8,49,0.6024,0.1643,0.2778,0.3735,0.5338,0.1873,0.8333,0.3919,OKC,GSW,0.6051,122.2753,91.6823,0.4761,13.5950,40.3459,0.3392,21.6928,25.1221,0.8625,9.7412,38.8787,24.6220,10.3161,10.0560,20.0371,16.3581,0.5508,0.0891,0.2193,0.2740,0.5024,0.1379,0.7873,0.2586,0.5870,1,1610612744,GSW,Golden State Warriors,GSW @ OKC,L,241,102,33,74,0.446,13,35,0.371,23,29,0.793,6,26,32,23,16,4,20,28,-24.0,228,49,56,52,28,8,49,0.5338,0.1873,0.1667,0.3919,0.6024,0.1643,0.7222,0.3735,OKC,0.3949,112.6197,88.5384,0.4455,14.3387,42.5288,0.3378,19.5079,24.7678,0.7988,11.2823,33.7009,26.5281,8.4533,13.9886,19.6012,3.7199,0.5272,0.1232,0.2414,0.2822,0.5227,0.1303,0.7645,0.2377,0.5221,0,1
1,0022500211,2025-11-11,22025,1610612762,UTA,Utah Jazz,UTA vs. IND,W,241,152,51,92,0.554,19,42,0.452,31,35,0.886,13,32,45,38,3,3,14,27,24.0,280,57,52,67,9,9,53,0.6576,0.1153,0.3939,0.3804,0.5152,0.0668,0.6809,0.2929,UTA,IND,0.5868,107.4835,90.2443,0.4056,12.1944,41.1549,0.2920,22.2075,27.3155,0.8103,14.6208,32.1523,27.1551,6.9709,16.1262,21.9327,-14.0903,0.4730,0.1361,0.2983,0.3051,0.5647,0.1172,0.7527,0.2745,0.4313,1,1610612754,IND,Indiana Pacers,IND @ UTA,L,240,128,46,99,0.465,10,37,0.270,26,29,0.897,15,20,35,29,6,6,8,26,-24.0,280,57,52,67,9,9,53,0.5152,0.0668,0.3191,0.2929,0.6576,0.1153,0.6061,0.3804,UTA,0.4132,100.6614,94.5405,0.3816,10.7283,37.7107,0.2838,17.8347,23.9560,0.7294,14.6640,34.7602,22.8710,7.1919,14.6127,23.5160,-15.0112,0.4383,0.1222,0.2782,0.2551,0.5227,0.1154,0.7539,0.3401,0.4071,0,1
2,0022500212,2025-11-11,22025,1610612758,SAC,Sacramento Kings,SAC vs. DEN,L,240,108,41,85,0.482,10,33,0.303,16,23,0.696,9,26,35,31,4,3,10,23,-14.0,230,37,59,55,10,5,42,0.5412,0.0951,0.2143,0.2706,0.5872,0.1092,0.6341,0.3140,SAC,DEN,0.4428,115.4702,88.6470,0.4664,12.0059,30.3964,0.3974,20.9260,26.3738,0.7764,9.2868,32.2401,26.3358,7.6551,14.5981,20.0546,-14.4707,0.5340,0.1269,0.2070,0.3000,0.5983,0.1070,0.7735,0.2648,0.4371,1,1610612743,DEN,Denver Nuggets,DEN @ SAC,W,239,122,44,86,0.512,13,33,0.394,21,27,0.778,15,33,48,24,6,2,12